# 第6章：构建真实 Taproot 合约> **参考**: `book/translations/zh-Hans/Chapter 6.md`  > **代码示例**: `code/chapter06/`  > **最后更新**: 2025-12-06---## 为什么 Script Path 改变一切

如果你能在 Bitcoin 上构建一个看起来像简单支付的合约——直到它揭示一个解锁资金的谜题，会怎样？

想象一下：你想创建一个数字悬赏，任何解决谜题的人都能获得奖励，但如果没有人解决，你想收回资金。或者你在销售数字商品，买家在支付后自动获得解锁密钥，但你保留退款控制权。

传统 Bitcoin 脚本要么完全暴露所有条件（损害隐私），要么需要复杂的多重签名设置（低效）。更糟糕的是，即使是简单的单签名支付也可以通过区块链上的交易模式轻松识别。

**Taproot 的 Script Path 使这一切变得优雅**：复杂的有条件合约在未花费时看起来与普通支付完全相同，只在需要时揭示必要的执行路径。

基于前几章建立的 Taproot 理论基础，我们现在深入探讨 Script Path 支出模式。本章通过实际的有条件支付场景演示如何在 Taproot 的脚本树中实现双路径授权：支持基于秘密的有条件支付和直接密钥持有者控制。

让我们从一个具体的业务场景开始，理解 Taproot Script Path 的实际价值：Alice 想创建一个具有以下特征的有条件支付合约：- **条件路径**：任何知道秘密词"helloworld"的人都可以认领资金- **替代路径**：Alice 作为资金所有者，可以随时使用她的私钥收回资金- **隐私要求**：未花费时，外部观察者无法区分简单支付和复杂合约这种双路径设计在实际应用中非常常见：| **应用场景** | **描述** ||-------------------------|----------------|| **数字商品销售** | 买家在支付后获得解锁密钥；卖家保留退款权限 || **悬赏任务** | 谜题解决者认领奖励；未使用的悬赏可由发布者收回 || **有条件托管** | 在特定条件下自动释放；否则所有者可以收回 || **教育激励** | 学生在正确答案时认领奖励；教师保留管理控制权 |

在我们的场景中，Alice 的 Taproot 地址包含两个花费路径：**Key Path（协作路径）**：- Alice 使用她的调整后私钥直接签名- 64 字节 Schnorr 签名，最大效率- 完全隐私，无脚本信息泄露**Script Path（脚本路径）**：- 使用哈希锁脚本：`OP_SHA256 <hash> OP_EQUALVERIFY OP_TRUE`- 任何知道原像"helloworld"的人都可以花费- 需要揭示脚本内容，但不暴露 Key Path 的存在

在深入代码实现之前，我们需要理解 Taproot 的核心开发模式：**Commit-Reveal**。这种模式将成为我们所有后续 Taproot 应用的基础开发模型。### Commit-Reveal 模式概述**Commit 阶段**：- 构建包含多个花费条件的脚本树- 将脚本树承诺到 Taproot 地址- 生成"中间地址"或"托管地址"以锁定资金- **外部方无法知道**具体花费条件**Reveal 阶段**：- 选择一个花费条件进行解锁- Key Path：使用密钥直接花费（最大隐私）- Script Path：揭示并执行特定脚本分支- **只暴露实际使用的条件**，其他条件永远保持私有这种模式的力量在于：在 Commit 阶段，所有不同复杂度的合约看起来完全相同；在 Reveal 阶段，只需要暴露实际使用的分支。

让我们通过简单的哈希锁案例学习 Taproot 单叶脚本的完整实现流程。基于前面介绍的 Alice 有条件支付场景，我们将实现：- **哈希锁脚本**：验证秘密词"helloworld"的哈希值- **单叶结构**：最简单的脚本树，仅包含一个脚本分支- **双路径支出**：Key Path（Alice 的直接控制）+ Script Path（有条件支出）### 什么是 Tagged Hash？在进入具体实现之前，我们需要理解一个重要概念：**Tagged Hash**。这是 BIP340 引入的哈希方法，为不同目的添加标签前缀：这种设计防止不同协议之间的哈希碰撞，大大增强安全性。每个标签都有其特定目的，确保来自不同上下文的哈希值永远不会意外重复。### 阶段 1：Commit 阶段——构建中间地址以锁定资金首先，Alice 需要将她的哈希锁脚本承诺到 Taproot 地址：**关键技术点分析：**1. **脚本序列化**：    ```    a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851    ```    - `a8`：OP_SHA256    - `20`：PUSH 32 字节    - `936a185c...07af`：SHA256("helloworld")    - `88`：OP_EQUALVERIFY    - `51`：OP_TRUE2. **TapLeaf 哈希计算**：    3. **输出密钥生成**：    **生成的中间地址**：`tb1p53ncq9ytax924ps66z6al3wfhy6a29w8h6xfu27xem06t98zkmvsakd43h`这是我们的**中间地址**或**托管地址**，资金被锁定在这里。对应的 ScriptPubKey：`OP_1 <32-byte-output-key>`，与区块链上任何其他 Taproot 地址完全不可区分。外部观察者无法区分这是简单的单签名还是复杂的有条件合约。### 阶段 2：Reveal 阶段——Key Path 支出（Alice 的直接控制）如果 Alice 决定直接收回资金，她可以使用 Key Path 支出：**Key Path 特征分析**：- **见证大小**：64 字节（固定大小的 Schnorr 签名）- **隐私级别**：完全隐私，无脚本信息泄露- **执行效率**：最高效率，单签名验证- **外观**：与任何简单 Taproot 支付完全相同**重要技术细节**：即使使用 Key Path，签名函数仍需要 `tapleaf_scripts` 参数来计算正确的调整值。这是因为 Taproot 的输出密钥是通过内部公钥和脚本树生成的。**Key Path 签名原理**：Alice 需要使用调整后的私钥签名以解锁资金。本质上，这也是恢复中间地址的过程。尽管 `script_path=False` 参数让库隐藏这些细节，但底层原理是：- **公钥线性对应**：`output_pubkey = internal_pubkey + tweak * G`- **私钥线性对应**：`tweak_private = internal_private + tweak`- **Schnorr 签名属性**：这种线性关系确保密钥对一致性这正是 Schnorr 签名相对于 ECDSA 的优势：支持密钥的线性组合和聚合。### 阶段 3：Reveal 阶段——Script Path 支出（有条件解锁）现在让我们实现 Script Path 支出的完整代码。与 Key Path 的简单性不同，Script Path 需要构建更复杂的见证数据来证明我们有使用特定脚本分支的权利。### Script Path 支出代码实现### 详细关键实现分析让我们深入探讨每个关键组件的实现原理：**1. 控制块（Control Block）构建****控制块结构图**：```Control Block Structure (33 bytes):┌─────────┬──────────────────────────────────┐│ Byte 1  │           Bytes 2-33             │├─────────┼──────────────────────────────────┤│   c1    │ 50be5fc4...126bb4d3             │├─────────┼──────────────────────────────────┤│Ver+Parity│         Internal Pubkey          │└─────────┴──────────────────────────────────┘Analysis:- c1 = c0 (leaf version) + 01 (parity flag)- Internal pubkey: Used to recalculate output key during verification```**技术要点**：- **内部公钥**：这是调整前的原始公钥，用于在验证期间重新计算输出密钥- **脚本树结构**：`[[tr_script]]` 表示只有一个叶子的脚本树，多个脚本将是 `[[script1], [script2]]`- **脚本索引**：标识在多脚本树中使用哪个脚本，对于单脚本始终为 0- **奇偶标志**：椭圆曲线点的 y 坐标可以是奇数或偶数，需要记录以进行完整重建**2. 见证数据的精确顺序****顺序重要性**：- Bitcoin Core 按固定顺序解析见证数据- 位置 `[n-1]`：控制块（最后一个元素）- 位置 `[n-2]`：脚本代码（倒数第二个）- 位置 `[0...n-3]`：脚本执行的输入参数**3. 原像的十六进制编码****编码原理**：- Bitcoin Script 处理十六进制字节数据- 字符串必须首先编码为 UTF-8 字节，然后转换为十六进制- 这确保脚本中的 OP_SHA256 可以正确处理输入数据### 实际交易执行结果分析运行上述代码后，我们可以在测试网上观察真实交易：**交易 ID**：`68f7c8f0ab6b3c6f7eb037e36051ea3893b668c26ea6e52094ba01a7722e604f`**见证数据分析**：```bashWitness Stack:[0] 68656c6c6f776f726c64                                                     # Preimage[1] a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851   # Script[2] c150be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3     # Control block```让我们验证这些数据的正确性：在验证脚本合法性和原像正确性后，我们还需要：**控制块验证** → 证明脚本在 Merkle 根中，**地址恢复验证** → 通过调整验证合法性，最后**栈执行** → 执行脚本逻辑。这些也是 Bitcoin Core 的验证步骤。### 控制块验证——证明脚本在 Merkle 树中### 地址恢复验证——通过调整验证合法性

如果遇到 Script Path 支出失败，请遵循此系统化调试方法：### 🔍 调试流程图："Script Path 支出失败？试试这个"**步骤 1：检查见证栈顺序**```✅ Correct Order: [preimage, script, control_block]❌ Wrong: [control_block, script, preimage]❌ Wrong: [script, preimage, control_block]```**步骤 2：验证脚本一致性**- Reveal 阶段的脚本是否与 Commit 阶段的脚本完全匹配？- 所有参数（哈希值、opcode）是否相同？- 两个阶段使用相同的 `build_hash_lock_script()` 函数**步骤 3：验证控制块**- 内部公钥是否正确？- 奇偶标志（`is_odd`）是否设置正确？ → 从 `taproot_address.is_odd()` 获取- 脚本索引是否匹配树位置？（单叶为 0）**步骤 4：检查输入数据编码**- 原像是否正确进行 UTF-8 编码然后转换为十六进制？- 示例：`"helloworld" → bytes → "68656c6c6f776f726c64"`**步骤 5：地址恢复测试**- 能否从内部密钥 + 脚本树重建相同的 Taproot 地址？- Commit 和 Reveal 阶段之间的调整计算是否匹配？### 1. 见证数据顺序错误**快速修复**：始终记住 Script Path 见证的"数据 → 代码 → 证明"顺序。### 2. 脚本序列化问题### 3. 控制块奇偶性错误### 4. Commit-Reveal 不一致

接下来，让我们观察哈希锁脚本的完整栈执行过程：**执行脚本**：`OP_SHA256 OP_PUSHBYTES_32 936a185c...07af OP_EQUALVERIFY OP_PUSHNUM_1`### 0. 初始栈状态：加载脚本输入```│ 68656c6c6f776f726c64                             ││ (preimage_hex: "helloworld")                     │└──────────────────────────────────────────────────┘```### 1. OP_SHA256：计算栈顶元素的 SHA256 哈希OP_SHA256 从栈顶弹出原像数据，计算其 SHA256 哈希，然后将结果推回栈：```│ 936a185caaa266bb9cbe981e9e05cb78cd732b0...                      ││ b3280eb944412bb6f8f8f07af                                       ││  (computed_hash)                                                │└─────────────────────────────────────────────────────────────────┘```**（计算过程：SHA256("helloworld") = 936a185c...07af）**### 2. PUSH 32 字节：推送预期哈希值脚本将预设的预期哈希值推送到栈顶：```│ 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af        │(expected_hash)                                                           ││ 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af        │ (computed_hash)                                                           │└─────────────────────────────────────────────────────────────────────────┘```**（栈中现在有两个相同的哈希值）**### 3. OP_EQUALVERIFY：验证哈希值相等OP_EQUALVERIFY 弹出栈顶两个元素进行比较，如果相等则继续执行，否则脚本失败：```│ (empty_stack) │└───────────────┘```**（验证成功：936a185c...07af == 936a185c...07af，两个元素都被消耗）**### 4. OP_TRUE：推送成功标志最后，OP_TRUE（OP_PUSHNUM_1）将值 1 推送到栈，标记脚本执行成功：```│ 01 (true_value) │└─────────────────┘```**（脚本执行成功：栈顶是非零值）**

通过实际代码实现和链上数据分析，我们可以清楚地看到两种支出方法之间的差异：| Aspect | Key Path | Script Path || --- | --- | --- || **Witness Data** | 1 element (64-byte signature) | 3 elements (input+script+control block) || **Transaction Size** | ~153 bytes | ~234 bytes || **Privacy Level** | Complete privacy, zero information leakage | Partial privacy, only exposes used script branch || **Verification Complexity** | Single Schnorr signature verification | Control block verification + script execution || **Fee Cost** | Lowest cost | Medium cost (~50% additional overhead) |这种**选择性揭示**设计使 Taproot 能够支持各种复杂的应用场景：数字商品销售、悬赏任务、有条件托管、多方合约等，同时在未使用时保持最大隐私。

与 P2SH 在花费时**所有条件都被揭示**不同，Taproot Script Path 确保只有执行的分支才会被看到。这种根本性转变重新定义了 Bitcoin 合约的隐私模型。**传统脚本局限性：**- 所有花费条件在链上可见- 复杂合约容易被观察者识别- 即使未使用的分支也损害隐私**Taproot 的隐私创新：**- 未使用的条件永远保持隐藏- 复杂合约与简单支付不可区分- 只揭示执行的逻辑，保持最大隐私这种隐私优先的设计使 Taproot 成为 Bitcoin 下一代智能合约的基础，其中复杂性不会损害机密性。

通过 Alice 的哈希锁合约案例，我们深入理解了 Taproot 对 Bitcoin 智能合约的革命性方法：### Commit-Reveal 模式的力量1. **Commit 阶段**：将复杂的有条件逻辑承诺到普通 Taproot 地址，生成中间地址以锁定资金2. **Reveal 阶段**：根据实际需要选择 Key Path 或 Script Path 支出，只暴露必要信息### 技术实现掌握1. **单叶脚本树**：TapLeaf 哈希直接作为 Merkle 根，无需额外的 Merkle 计算2. **控制块验证**：通过地址恢复密码学证明脚本合法性3. **栈执行**：哈希锁通过哈希匹配验证实现有条件支出### 开发最佳实践1. **Tagged Hash 理解**：掌握不同目的的哈希标签，确保安全性2. **见证数据顺序**：严格遵循 [输入参数, 脚本, 控制块] 顺序3. **系统化调试**：使用我们的调试流程图排查常见问题4. **代码一致性**：使用辅助函数确保 commit-reveal 阶段对齐### 更大的图景本章建立的不仅仅是哈希锁实现——它展示了 Taproot 的根本性隐私革命。通过允许复杂合约在执行之前伪装成简单支付，Taproot 在不牺牲用户隐私的情况下改变了 Bitcoin 的能力。**下一步**：在下一章中，我们将探索**双叶脚本树**，学习如何在一个 Taproot 地址中组织多个不同的花费条件，引入真实的 Merkle 树计算，并体验 Taproot 脚本树架构的完整力量，用于更复杂的应用场景。